# 16Q_CR_Square_HighCoupling

**Description:** 16-qubit processor in a 4x4 square lattice, optimized for cross-resonance (CR) gates using fixed-frequency transmons and fixed capacitive coupling, with increased coupling strength.

Generated by QFoundry AI Agent.

In [ ]:
%matplotlib inline
import sys, os
import numpy as np
import pandas as pd
import rustworkx as rx
from rustworkx.visualization import mpl_draw
from matplotlib import pyplot as plt

from qfoundry import cpw_resonator, cpw
from qfoundry.utils import sc_metal
from qfoundry.qubits import transmon, tunable_transmon
from qfoundry.PDK import PDK
from qfoundry.topology import LatticeGenerator
from qfoundry.coloring import RelaxedColoring

pdk = PDK()
print(f"Loaded PDK: {pdk}")

In [ ]:
# Design Specifications
design_name = '16Q_CR_Square_HighCoupling'
n_qubits = 16
topology_type = 'square'
grid_rows = 4
grid_cols = 4
coloring_strategy_name = 'RelaxedColoring'

# Qubit Families Parameters
qubit_families = [
    {'family_id': 'FamilyA', 'qubit_type': 'transmon', 'frequency_mean': 4800000000.0, 'anharmonicity': -250000000.0, 'asymmetry_max': 0.05, 'charging_energy': 250000000.0, 'drive_line': True, 'drive_coupling': 30000000.0, 'readout_resonator': True, 'resonator_coupling': 10000000.0, 'color': 'blue'},
    {'family_id': 'FamilyB', 'qubit_type': 'transmon', 'frequency_mean': 5000000000.0, 'anharmonicity': -250000000.0, 'asymmetry_max': 0.05, 'charging_energy': 250000000.0, 'drive_line': True, 'drive_coupling': 30000000.0, 'readout_resonator': True, 'resonator_coupling': 10000000.0, 'color': 'red'},
    {'family_id': 'FamilyC', 'qubit_type': 'transmon', 'frequency_mean': 5200000000.0, 'anharmonicity': -250000000.0, 'asymmetry_max': 0.05, 'charging_energy': 250000000.0, 'drive_line': True, 'drive_coupling': 30000000.0, 'readout_resonator': True, 'resonator_coupling': 10000000.0, 'color': 'green'},
    {'family_id': 'FamilyD', 'qubit_type': 'transmon', 'frequency_mean': 5400000000.0, 'anharmonicity': -250000000.0, 'asymmetry_max': 0.05, 'charging_energy': 250000000.0, 'drive_line': True, 'drive_coupling': 30000000.0, 'readout_resonator': True, 'resonator_coupling': 10000000.0, 'color': 'purple'},
]

# Guidelines
guidelines = [
    "Qubit frequencies must be allocated across the 4x4 square lattice using the 'RelaxedColoring' strategy with the defined qubit families to ensure sufficient spectral separation between nearest neighbors (target >= 200 MHz) for reduced crosstalk and effective cross-resonance operation.",
    "Each qubit requires an individual microwave drive line for single-qubit gate operations and applying the drive pulses for cross-resonance gates.",
    "Each qubit must be coupled to a dedicated readout resonator, with resonators designed for distinct frequencies and sufficient detuning from their respective qubits and other resonators to enable fast, high-fidelity multiplexed readout.",
    "The fixed capacitive coupling between qubits is designed to be always-on, providing the necessary strong interaction for cross-resonance gates without requiring active tuning of the coupling element. Given the increased coupling strength of 150 MHz, the qubit frequencies are allocated with at least 200 MHz detuning between nearest neighbors to minimize unwanted direct resonant interactions, while still enabling efficient cross-resonance operations."
]

In [ ]:
# Generate Topology
lattice_gen = LatticeGenerator()

if topology_type == 'square':
    graph, info = lattice_gen.generate_square_lattice(grid_rows, grid_cols)
elif topology_type == 'hexagonal':
    graph, info = lattice_gen.generate_hexagonal_lattice(grid_rows, grid_cols)
else:
    # Fallback to square
    print(f"Warning: Unknown topology {topology_type}, defaulting to square.")
    graph, info = lattice_gen.generate_square_lattice(grid_rows, grid_cols)

print(f"Generated {info.lattice_type} lattice with {info.num_nodes} nodes and {info.num_edges} edges.")

# Visualize
plt.figure(figsize=(8, 8))
fig = rx.visualization.mpl_draw(graph, pos=info.node_positions, with_labels=True)
plt.title(f"{design_name} Topology")
plt.show()

In [ ]:
# Assign Frequencies based on Families using Coloring Strategy

# Prepare families dict for coloring function
available_families = {}
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b'] # matplotlib default cycle
for i, fam in enumerate(qubit_families):
    available_families[fam['family_id']] = {
        'color': colors[i % len(colors)],
        'frequency': fam['frequency_mean']
    }

# Select Strategy
if coloring_strategy_name == 'RelaxedColoring':
    strategy = RelaxedColoring(max_same_color_neighbors=2)
    color_map = strategy(graph)
else:
    # Use rustworkx built-in greedy coloring
    color_map = rx.graph_greedy_color(graph)

print(f"Applying coloring strategy: {coloring_strategy_name}")

node_frequencies = {}
num_families = len(qubit_families)

for node in graph.node_indices():
    color_idx = color_map[node]
    family_idx = color_idx % num_families
    fam_spec = qubit_families[family_idx]
    node_frequencies[node] = fam_spec['frequency_mean']
    
print("Assigned Frequencies:")
print(node_frequencies)

# Visualize with Frequencies
plt.figure(figsize=(10, 8))
node_colors = [node_frequencies[n] for n in graph.node_indices()]
rx.visualization.mpl_draw(
    graph, 
    pos=info.node_positions, 
    with_labels=True, 
    node_color=node_colors, 
    cmap='viridis',
    node_size=500
)
plt.title(f"{design_name} - Frequency Allocation ({coloring_strategy_name})")
plt.colorbar(plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(min(node_colors), max(node_colors))), label='Frequency (Hz)')
plt.show()

# Print detailed family info including drive/readout
print("\nDetailed Qubit Family Specifications:")
for fam in qubit_families:
    print(f"Family {fam['family_id']}:")
    print(f"  Type: {fam.get('qubit_type', 'N/A')}")
    print(f"  Freq: {fam['frequency_mean']/1e9:.2f} GHz")
    if fam.get('drive_line'):
        print(f"  Drive Line: Yes (Coupling: {fam.get('drive_coupling', 0)/1e6:.1f} MHz)")
    else:
        print(f"  Drive Line: No")
        
    if fam.get('readout_resonator'):
        print(f"  Readout Resonator: Yes (Coupling: {fam.get('resonator_coupling', 0)/1e6:.1f} MHz)")
    else:
        print(f"  Readout Resonator: No")